### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [ ]:
reviews['date'] = pd.to_datetime(reviews['date'])

#Create flag that warns less than 5 reviews

number_flag = reviews.groupby('movie_id')['rating'].count().reset_index()
number_flag['flag'] = np.where(number_flag['rating']< 5, 1, 0)
reviews = reviews.merge(number_flag[['movie_id','flag']], how = 'left', left_on = 'movie_id', right_on = 'movie_id')

#rank movies by the most recent review 

rank_by_recency = reviews.groupby(['movie_id'],
                                  sort = False)['date'].max().sort_values(ascending = False).rank(method = "min",
                                                                                                  ascending = False)
rank_by_recency = rank_by_recency.to_frame(name = 'rank_by_recency')
reviews = reviews.merge(rank_by_recency, how = 'left', left_on = 'movie_id', right_on = 'movie_id')

#rank movies by the number of reviews
rank_by_count = reviews.groupby('movie_id').count().sort_values(by = ['rating'], ascending = False)['rating'].rank(method = "min", ascending = False)
rank_by_count = rank_by_count.to_frame(name = 'rank_by_count')
reviews = reviews.merge(rank_by_count, how = 'left', left_on = 'movie_id', right_on = 'movie_id')

#rank movies by the average rating
rank_by_average = reviews.groupby('movie_id').mean().sort_values(by = ['rating'], ascending = False)['rating'].rank(method = "min", ascending = False)
rank_by_average = rank_by_average.to_frame(name = 'rank_by_average')
reviews = reviews.merge(rank_by_average, how = 'left', left_on = 'movie_id', right_on = 'movie_id')

In [ ]:
movies.head(2)

In [ ]:
movies = movies.merge(reviews[['movie_id', 'flag', 'rank_by_count', 'rank_by_average', 'rank_by_recency']].drop_duplicates(subset=['movie_id']),
                      how = 'left', left_on = 'movie_id', right_on = 'movie_id')

In [ ]:
movies.head()

In [ ]:
top_movies = movies.sort_values(by = ['flag', 'rank_by_average', 'rank_by_count', 'rank_by_recency'], ascending = True)['movie'].head(2)

In [ ]:
list(top_movies)

In [ ]:
movies['movie_id'].nunique()

In [4]:
def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Read in the datasets
    movies = pd.read_csv('movies_clean.csv')
    reviews = pd.read_csv('reviews_clean.csv')
    del movies['Unnamed: 0']
    del reviews['Unnamed: 0']
    
    
    reviews['date'] = pd.to_datetime(reviews['date'])

    #Create flag that warns less than 5 reviews

    number_flag = reviews.groupby('movie_id')['rating'].count().reset_index()
    number_flag['flag'] = np.where(number_flag['rating']< 5, 1, 0)
    reviews = reviews.merge(number_flag[['movie_id','flag']], how = 'left', left_on = 'movie_id', right_on = 'movie_id')

    #rank movies by the most recent review 

    rank_by_recency = reviews.groupby(['movie_id'],
                                      sort = False)['date'].max().sort_values(ascending = False).rank(method = "min",
                                                                                                  ascending = False)
    rank_by_recency = rank_by_recency.to_frame(name = 'rank_by_recency')
    reviews = reviews.merge(rank_by_recency, how = 'left', left_on = 'movie_id', right_on = 'movie_id')

    #rank movies by the number of reviews
    rank_by_count = reviews.groupby('movie_id').count().sort_values(by = ['rating'], ascending = False)['rating'].rank(method = "min", ascending = False)
    rank_by_count = rank_by_count.to_frame(name = 'rank_by_count')
    reviews = reviews.merge(rank_by_count, how = 'left', left_on = 'movie_id', right_on = 'movie_id')

    #rank movies by the average rating
    rank_by_average = reviews.groupby('movie_id').mean().sort_values(by = ['rating'], ascending = False)['rating'].rank(method = "min", ascending = False)
    rank_by_average = rank_by_average.to_frame(name = 'rank_by_average')
    reviews = reviews.merge(rank_by_average, how = 'left', left_on = 'movie_id', right_on = 'movie_id')
    
    movies = movies.merge(reviews[['movie_id', 'flag', 'rank_by_count', 'rank_by_average', 'rank_by_recency']].drop_duplicates(subset=['movie_id']),
                          how = 'left', left_on = 'movie_id', right_on = 'movie_id')
    
    top_movies = movies.sort_values(by = ['flag', 'rank_by_average', 'rank_by_count', 'rank_by_recency'], ascending = True)['movie'].head(n_top)
    
    return list(top_movies) # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [5]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations(1, 20) # Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968, 5) # Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(70000, 100) # Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(43, 35) # Your solution list here



In [6]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [ ]:
t.popular_recs_filtered('1', 20, movies, years=['2015', '2016', '2017', '2018'], genres=['History', 'News', 'Horror'])

In [ ]:
movies.head(2)

In [4]:
years=['2015', '2016', '2017', '2018']
genres=['History', 'News']

In [9]:
movies_filtered = movies[movies['date'].isin(years) & movies[genres].any(1)]

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
13156,420293,The Stanford Prison Experiment (2015),Biography|Drama|History,2015,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
14259,490215,Silence (2016),Drama|History,2016,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
15853,1083452,Eddie the Eagle (2016),Biography|Drama|History,2016,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
16103,1135989,A Tale of Love and Darkness (2015),Biography|Drama|History,2015,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
16753,1254322,The Girl King (2015),Biography|Drama|History,2015,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
17447,1398426,Straight Outta Compton (2015),Biography|Drama|History,2015,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
17522,1413492,12 Strong (2018),Action|Drama|History,2018,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,0,0
18968,1668016,A Real Vermeer (2016),Drama|History|Romance,2016,0,0,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
19145,1698654,Catching the Sun (2015),Documentary|Drama|History,2015,0,0,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0
19228,1710898,42nd Street Memories: The Rise and Fall of Ame...,History,2015,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
movies.shape

(31245, 35)

In [21]:
years = list(movies['date'].unique())
movies[movies['date'].isin(years)]

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
5,417,Le voyage dans la lune (1902),Short|Adventure|Fantasy,1902,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
6,439,The Great Train Robbery (1903),Short|Action|Crime,1903,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
7,443,"Hiawatha, the Messiah of the Ojibway (1903)",NaN,1903,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,628,The Adventures of Dollie (1908),Action|Short,1908,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
9,833,The Country Doctor (1909),Short|Drama,1909,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [12]:
list(movies['date'].unique())

[1894,
 1895,
 1896,
 1902,
 1903,
 1908,
 1909,
 1910,
 1911,
 1912,
 1913,
 1914,
 1915,
 1916,
 1917,
 1918,
 1919,
 1920,
 1921,
 1922,
 1923,
 1924,
 1925,
 1926,
 1927,
 1928,
 1929,
 1930,
 1931,
 1932,
 1933,
 1934,
 1935,
 1936,
 1937,
 1940,
 1938,
 1939,
 1941,
 1942,
 1944,
 1943,
 2001,
 1945,
 1946,
 1947,
 1948,
 1949,
 1952,
 1950,
 1951,
 1954,
 1953,
 1955,
 1956,
 1957,
 1958,
 1959,
 1961,
 1962,
 1960,
 1963,
 1964,
 1965,
 1967,
 1966,
 1969,
 1968,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1978,
 1977,
 1981,
 1979,
 1982,
 1980,
 1989,
 1986,
 1983,
 1984,
 1987,
 1985,
 1988,
 1990,
 1992,
 1991,
 1993,
 1994,
 1995,
 1996,
 1999,
 1997,
 1998,
 2005,
 2000,
 2003,
 2002,
 2010,
 2004,
 2008,
 1901,
 2006,
 2007,
 2012,
 1900,
 2009,
 2016,
 2011,
 2015,
 2017,
 2013,
 2018,
 2014,
 1888,
 1898,
 1878,
 1907]

In [26]:
def popular_recommendations_filtered(user_id, n_top, years=None, genres=None):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Read in the datasets
    movies = pd.read_csv('movies_clean.csv')
    reviews = pd.read_csv('reviews_clean.csv')
    del movies['Unnamed: 0']
    del reviews['Unnamed: 0']
    
    
    #reviews['date'] = pd.to_datetime(reviews['date'])
    
    #If there are no filters, we "filter" on all values
    if years == None:
        years = list(movies['date'].unique())
    if genres == None:
        genres = movies.columns[7:]
    
    #filter the dataset
    movies_filtered = movies[movies['date'].isin(years) & movies[genres].any(1)]
    
    #Create flag that warns less than 5 reviews

    number_flag = reviews.groupby('movie_id')['rating'].count().reset_index()
    number_flag['flag'] = np.where(number_flag['rating']< 5, 1, 0)
    reviews = reviews.merge(number_flag[['movie_id','flag']], how = 'left', left_on = 'movie_id', right_on = 'movie_id')

    #rank movies by the most recent review 

    rank_by_recency = reviews.groupby(['movie_id'],
                                      sort = False)['date'].max().sort_values(ascending = False).rank(method = "min",
                                                                                                  ascending = False)
    rank_by_recency = rank_by_recency.to_frame(name = 'rank_by_recency')
    reviews = reviews.merge(rank_by_recency, how = 'left', left_on = 'movie_id', right_on = 'movie_id')

    #rank movies by the number of reviews
    rank_by_count = reviews.groupby('movie_id').count().sort_values(by = ['rating'], ascending = False)['rating'].rank(method = "min", ascending = False)
    rank_by_count = rank_by_count.to_frame(name = 'rank_by_count')
    reviews = reviews.merge(rank_by_count, how = 'left', left_on = 'movie_id', right_on = 'movie_id')

    #rank movies by the average rating
    rank_by_average = reviews.groupby('movie_id').mean().sort_values(by = ['rating'], ascending = False)['rating'].rank(method = "min", ascending = False)
    rank_by_average = rank_by_average.to_frame(name = 'rank_by_average')
    reviews = reviews.merge(rank_by_average, how = 'left', left_on = 'movie_id', right_on = 'movie_id')
    
    movies_filtered = movies_filtered.merge(reviews[['movie_id', 'flag', 'rank_by_count', 'rank_by_average', 'rank_by_recency']].drop_duplicates(subset=['movie_id']),
                          how = 'left', left_on = 'movie_id', right_on = 'movie_id')
    
    top_movies = movies_filtered.sort_values(by = ['flag', 'rank_by_average', 'rank_by_count', 'rank_by_recency'], ascending = True)['movie'].head(n_top)
    
    return list(top_movies) # a list of the n_top movies as recommended

In [27]:
popular_recommendations_filtered(1, 20)

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Crawl Bitch Crawl (2012)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Third Contact (2011)',
 'Romeo Juliet (2009)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Agnelli (2017)',
 'Sátántangó (1994)',
 'Shijie (2004)',
 'Foster (2011)',
 'CM101MMXI Fundamentals (2013)',
 'Akahige (1965)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
 'Körkarlen (1921)']

In [28]:
t.popular_recs_filtered('1', 20, ranked_movies)

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Crawl Bitch Crawl (2012)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Third Contact (2011)',
 'Romeo Juliet (2009)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Agnelli (2017)',
 'Sátántangó (1994)',
 'Shijie (2004)',
 'Foster (2011)',
 'CM101MMXI Fundamentals (2013)',
 'Akahige (1965)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
 'Körkarlen (1921)']